In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os 
import glob
from FlowCytometryTools import FCMeasurement
from collections import Counter
import pdb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import xlrd
from collections import Counter
from sklearn import preprocessing

In [ ]:
# set the path!
path = '/Users/joannaf/Desktop/courses/DeepLearning/DL2019/project/data/Dataset5/'


### Preprocessing of Chevrier dataset
This noteboook provides codes for preprocessing of the dataset described in 
https://www.sciencedirect.com/science/article/pii/S0092867417304294?via%3Dihub#mmc2 . 
The files downloaded from the cytobank were passed thorugh chevrier_correct_fcs.R script,
since they were corrupted and it was not possible to load them directly into jupyter notebook. The cluster identity (cell-types) were obtained by reading out information from figure 2f (cell-type identity of the clusters provided with the data).



In [2]:
### cell-type information was read out from Figure 2f in the paper:
# TAM panel
tam_cluster_identity = dict()
tam_cluster_identity['22'] = 'pDC'
tam_cluster_identity['6'] = 'NK cells'
tam_cluster_identity['12'] = 'B cells'
tam_cluster_identity['14'] = 'Unknown'
tam_cluster_identity['16'] = 'Plasma cells'
tam_cluster_identity['20'] = 'DC'
for x in ['24','18','11','21','25','3','10']:
    tam_cluster_identity[x] = 'Non immune cells'
for x in ['5','1','8']:
    tam_cluster_identity[x] = 'Myeloid cells'
for x in ['4','13','9','0','15','23','19','17','2','7']:
    tam_cluster_identity[x] = 'T cells'

# Tcell panel
tcell_cluster_identity = dict()
tcell_cluster_identity['23'] = 'pDC'
tcell_cluster_identity['5'] = 'NK cells'
tcell_cluster_identity['18'] = 'Plasma cells'
tcell_cluster_identity['15'] = 'B cells'
tcell_cluster_identity['22'] = 'Granulocytes'
for x in ['28','17','16','25','14','4','21','10','20']:
    tcell_cluster_identity[x] = 'Non immune cells'
for x in ['26','27','6','1','11']:
    tcell_cluster_identity[x] = 'Myeloid cells'
for x in ['9','29','19','3','7','0','13','24','8','2','12']:
    tcell_cluster_identity[x] = 'T cells'


In [3]:
data_dir=path+'data_corrected/'
files = []
for root, directories, filenames in os.walk(data_dir):
    for file in filenames:
        if '.fcs' in file:
            files.append(os.path.join(root, file))
print(len(files))            
file_info = dict()
for f in files:
    idx = f.split(data_dir)[-1]
    file_info[idx] = dict()
    file_info[idx]['sample'] = f.split('/')[-1].split('.')[0]
    file_info[idx]['experiment'] = f.split('/')[-2]
    df = FCMeasurement(ID='Sample', datafile=f)
    file_info[idx]['ncells'] = df.data.shape[0]
    file_info[idx]['nchannels'] = df.data.shape[1]
    file_info[idx]['channels'] = [x for x in df.data.columns if 'Di' in x]

308


/Users/joannaf/anaconda3/envs/dl/lib/python3.7/site-packages/fcsparser/api.py:332: UserWarning: The default channel names (defined by the $PnS parameter in the FCS file) were not unique. To avoid problems in downstream analysis, the channel names have been switched to the alternate channel names defined in the FCS file. To avoid seeing this warning message, explicitly instruct the FCS parser to use the alternate channel names by specifying the channel_naming parameter.
  warnings.warn(msg)


In [4]:
# summarize the information about all files
summary_df = pd.DataFrame(index=file_info[list(file_info.keys())[0]].keys(),
                           columns=file_info.keys())

for f in file_info.keys():
    summary_df.loc[:,f] = pd.DataFrame.from_dict(file_info[f], orient='index')
summary_df = summary_df.transpose()
summary_df.head(2)


,sample,experiment,ncells,nchannels,channels
experiment_102007_files/rcc7.fcs,rcc7,experiment_102007_files,11842,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P..."
experiment_102007_files/rcc17.fcs,rcc17,experiment_102007_files,14,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P..."


In [5]:
### clinical info
fname = 'clinical_info.xlsx'
xl_file = pd.ExcelFile(path+fname)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
df_clinical = pd.read_excel(path+fname, sheet_name='Sheet3')
df_batch = pd.read_excel(path+fname, sheet_name='Sheet1')
df_clinical = df_clinical.merge(df_batch, right_on='Sample ID', 
                                left_on=df_clinical.columns[0])

# add clinical info to the summary_df
df_clinical['sample'] = ['rcc'+str(x) for x in df_clinical.iloc[:,0]]
summary_df = summary_df.merge(df_clinical, on='sample')
summary_df.head(2)
### excluded samples: rcc61 (outlier for myeloid compartment); 
# rcc25, rcc17 (insufficient immune cell number)

,sample,experiment,ncells,nchannels,channels,Code used in the file name,Tissue Bank ID#,Subtype,Gender,Age,...,Date Last Follow-up,Status (Alive/Dead),Date of Death,Time to death,Length FU (yr),Cause Death,Case Note,Sample ID,Site,Grade_y
0,rcc7,experiment_102007_files,11842,55,"[beadDist, Y89Di, Pd102Di, Pd104Di, Pd105Di, P...",7,62224,ccRCC,M,57,...,2015-09-10,A,NaT,NaN,NaN,NaN,NaN,7,UHN,3
1,rcc7,experiment_101718_files,30365,55,"[Y89Di, Pd102Di, Pd104Di, Pd105Di, Pd106Di, Pd...",7,62224,ccRCC,M,57,...,2015-09-10,A,NaT,NaN,NaN,NaN,NaN,7,UHN,3


In [6]:
### antibodies (according to the description and supplementary material)
fname = 'antibodies.xlsx'
xl_file = pd.ExcelFile(path+fname)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
#display(dfs)
df_anti_tam = pd.read_excel(path+fname, sheet_name='Sheet1')
df_anti_tcell = pd.read_excel(path+fname, sheet_name='Sheet2')

anti_tam = dict(zip([x+'Di' for x in df_anti_tam.loc[:,'Metal Tag']],
                    df_anti_tam.loc[:,'Target']))

anti_tcell = dict(zip([x+'Di' for x in df_anti_tcell.loc[:,'Metal Tag']],
                    df_anti_tcell.loc[:,'Target']))

# CD86, CD20, CD206, CD68, and CD15 were excluded from the analysis to remove markers
# not expressed on T cells and likely to add noise in the cluster generation process.
# CD7 was also excluded from the PhenoGraph clustering since this marker split most clusters
# into CD7+ and CD7- fractions without a clear biological meaning and simultaneously reduced
# the impact of more biologically relevant markers on the clustering

In [7]:
### combine files (rbind) per experiment
experiments = list(set(summary_df['experiment']))

data_series = dict()#pd.Series({})
for exp in experiments:
    files_exp = [x for x in files if exp in x]
    exp_data_series = dict()
    for f in files_exp:
        name = f.split('/')[-1].split('.')[0]
        exp_data_series[name] = FCMeasurement(ID='Sample', datafile=f).data
    data_series[exp] = pd.concat(exp_data_series, axis=0)
    

In [8]:
### correct the names across experiments and add the cell-type information
col_names = []
data_series_pp = dict()
for key in data_series.keys():
    df = data_series[key]
    col_tmp = df.columns
    if(len(col_tmp)<=55):
        anti_dict = anti_tam.copy()
        panel = "tam_panel"
        cell_type_dict = tam_cluster_identity
    else:
        anti_dict = anti_tcell.copy()
        panel = "tcell_panel"
        cell_type_dict = tcell_cluster_identity
    anti_dict['phenograph'] = 'metadata_phenograph'
    anti_dict['PhenoGraph'] = 'metadata_phenograph'
    for t in col_tmp:
        if(t not in anti_dict.keys()):
            anti_dict[t] = 'metadata_'+t
    col_new = [anti_dict[x] for x in col_tmp]
    df.columns = col_new
    df['metadata_panel'] = panel
    df['metadata_celltype'] = [cell_type_dict[str(int(x))] for x in df['metadata_phenograph']]
    col_names.extend(df.columns)
    data_series[key] = df
col_names_count = Counter(col_names)


In [26]:
# merge data across experiments
data_all_merged = pd.concat(data_series, axis=0)
data_all_merged['metadata_sample'] = data_all_merged.index.get_level_values(1)
data_all_merged['metadata_batch'] = data_all_merged.index.get_level_values(0)
# remove samples 61,25 and 17, as indicated in the manuscripts
data_all_merged = data_all_merged.drop(labels = ['rcc61','rcc25','rcc17'],axis=0, level=1)
data_all_merged.shape

/Users/joannaf/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(4705750, 92)

In [27]:
### create an index merging the important information
#(for compatibility with other parts of the pipeline)
batch_unique = list(set(data_all_merged['metadata_batch']))
batch_dict = dict(zip(batch_unique, ['batch'+str(x+1) for x in range(len(batch_unique))]))
sample_unique = list(set(data_all_merged['metadata_sample']))
sample_dict = dict(zip(sample_unique, ['sample'+str(x+1) for x in range(len(sample_unique))]))

batch_renamed = [batch_dict[x] for x in data_all_merged['metadata_batch']]
data_all_merged['metadata_experiment'] = data_all_merged['metadata_batch']
data_all_merged['metadata_batch'] = batch_renamed
sample_renamed = [sample_dict[x] for x in data_all_merged['metadata_sample']]
sample_renamed = [sample_dict[x] for x in data_all_merged['metadata_sample']]
data_all_merged['metadata_sample_org'] = data_all_merged['metadata_sample']
data_all_merged['metadata_sample'] = sample_renamed
panel_renamed = [x.replace('_','.') for x in data_all_merged['metadata_panel']]
celltype_renamed = ['celltype'+x.replace(' ','.') for x in data_all_merged['metadata_celltype']]
idx = ['_'.join([a,b,c,d]) for a,b,c,d in zip(batch_renamed, sample_renamed, panel_renamed,celltype_renamed)]
data_all_merged.index = idx

In [29]:
### preprocessing steps as described in the paper
def arcsinh(x, c=5):
    if(x is not np.nan):
        x = np.log(x/c + np.sqrt((x/c)**2 +1))
    return(x)

data_all_merged = data_all_merged.apply(lambda x: arcsinh(x) if 'metadata' not in x.name else x)


In [30]:
data_all_merged.head(2)

,CCR7,CD119 (IFN-g R a chain),CD11b,CD11c,CD123,CD127,CD13,CD134,CD14,CD15,...,metadata_Yb170Di,metadata_barcode,metadata_beadDist,metadata_celltype,metadata_panel,metadata_phenograph,metadata_sample,metadata_batch,metadata_experiment,metadata_sample_org
batch1_sample5_tam.panel_celltypeNK.cells,NaN,0.106002,0.173622,NaN,0.0,NaN,0.000000,NaN,0.009175,NaN,...,NaN,4.0,69.986053,NK cells,tam_panel,6.0,sample5,batch1,experiment_101718_files,rcc7
batch1_sample5_tam.panel_celltypeNon.immune.cells,NaN,0.215915,0.270593,NaN,0.0,NaN,0.404903,NaN,0.000000,NaN,...,NaN,4.0,69.775345,Non immune cells,tam_panel,11.0,sample5,batch1,experiment_101718_files,rcc7


In [31]:
table = pa.Table.from_pandas(data_all_merged)
pq.write_table(table, path+'chevrier_data_pooled_panels.parquet')


In [91]:
cell_per_group = data_all_merged.groupby(['metadata_sample', 'metadata_panel']).metadata_batch.value_counts()

metadata_panel  metadata_batch
tam_panel       batch1            65150
                batch4             9106
tcell_panel     batch3            73100
                batch2            49059
Name: metadata_batch, dtype: int64

In [104]:
cell_per_group['sample14']

metadata_panel  metadata_batch
tam_panel       batch1            11306
                batch4             2606
tcell_panel     batch3             9758
                batch2             2127
Name: metadata_batch, dtype: int64

In [98]:
#cell_per_group.iloc[0:50]
sample_ncell = data_all_merged.groupby(['metadata_sample']).metadata_barcode.count().sort_values()
dict(zip(sample_ncell.index, sample_ncell))

{'sample1': 1703,
 'sample67': 2485,
 'sample40': 2647,
 'sample12': 4437,
 'sample36': 5006,
 'sample47': 5157,
 'sample26': 6036,
 'sample71': 9308,
 'sample13': 9914,
 'sample3': 13983,
 'sample16': 14936,
 'sample51': 16572,
 'sample54': 19014,
 'sample75': 19970,
 'sample37': 22516,
 'sample45': 23892,
 'sample9': 24024,
 'sample19': 24786,
 'sample70': 24885,
 'sample14': 25797,
 'sample15': 28069,
 'sample61': 28451,
 'sample35': 33661,
 'sample42': 36122,
 'sample34': 37140,
 'sample33': 37709,
 'sample48': 37980,
 'sample62': 38341,
 'sample43': 39691,
 'sample30': 40002,
 'sample68': 40064,
 'sample69': 40392,
 'sample55': 40925,
 'sample73': 43232,
 'sample46': 43742,
 'sample29': 44453,
 'sample6': 44530,
 'sample7': 46356,
 'sample17': 50018,
 'sample11': 50227,
 'sample4': 50553,
 'sample32': 52641,
 'sample65': 54701,
 'sample22': 56566,
 'sample28': 58234,
 'sample8': 59476,
 'sample25': 59909,
 'sample38': 61546,
 'sample56': 61687,
 'sample39': 61748,
 'sample23': 628

In [ ]:
############################   sanity checks  ##############################

In [ ]:
### sanity checks
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import umap
reducer = umap.UMAP()
import matplotlib.cm as cm
import os

def plot_tsne(data, do_pca=True, n_plots=2, iter_=500, pca_components=20, save_as=None, folder_name='figures'):
    ''' 
    Function to generate t-sne plot 
    inputs: 
        data: cell x markers: has the labels as index!! eg. Data23_Panel1_tx_NR4_Patient9
        do_pca: performs pca prior to t-sne, no downsampling there
        n_plots: Tries different perplexity values, 
        iter_ : fitting 
        pca_components: PCs
    '''
    Labels = list(data.index)
    if do_pca: 
        pca = PCA(n_components=pca_components)
        data = pca.fit_transform(data)
    for i in range(n_plots):
        perplexity_ = 10* (i + 1)
        tsne = TSNE(n_components=2,verbose=1,perplexity=perplexity_,n_iter=iter_)
        X_tsne = tsne.fit_transform(data)
        Xf = pd.DataFrame(X_tsne)
        Xf.columns = ["t-sne1", "t-sne2"]
        Xf['labels'] = Labels
        sns.lmplot("t-sne1", "t-sne2",hue="labels",data=Xf, fit_reg=False, scatter_kws={'alpha': 0.1})
        plt.title('Plot: t-SNE projection of the dataset perplexity = {}, iter = {}'.format(perplexity_, iter_), fontsize=15)
        if save_as is not None:
            plt.savefig(os.path.join(folder_name, save_as+'_p'+str(perplexity_)))
            plt.close()
        else:
            plt.show()

#min_max_scaler = preprocessing.MinMaxScaler()
def scale(x):
    p99 = np.percentile(x,99)
    x[x>p99] = p99
    x = (x - np.min(x)) / (np.max(x) - np.min(x))
    return(x)

def prep_for_tsne(data, max_cells=2000, random_state=345):
    df = data.copy()
    np.random.seed(random_state)
    df['metadata_number'] = range(df.shape[0])
    selected_cells = df.loc[:,['metadata_celltype','metadata_number']].groupby('metadata_celltype',group_keys=False).apply(lambda x: x.sample(min(len(x),max_cells)))
    df = df.iloc[selected_cells['metadata_number'],:]
    df = df.loc[:,~df.columns.str.startswith('metadata')]
    cts = [x.split('_')[-1].split('celltype')[-1] for x in list(df.index)]
    df.index = cts
    df = df.apply(lambda x: scale(x), axis=0)
    return(df)


In [ ]:
tam_df = data_all_merged.loc[data_all_merged['metadata_panel']=='tam_panel',:]#~data_all_merged.columns.str.startswith('metadata')]
print(tam_df.shape)
tam_df = tam_df.dropna(axis=1)
tam_df = prep_for_tsne(tam_df)
print(tam_df.shape)

tcell_df = data_all_merged.loc[data_all_merged['metadata_panel']=='tcell_panel',:]#~data_all_merged.columns.str.startswith('metadata')]
print(tcell_df.shape)
tcell_df = tcell_df.dropna(axis=1)
tcell_df = prep_for_tsne(tcell_df)
print(tcell_df.shape)

In [ ]:
plot_tsne(tam_df, n_plots=1)

In [ ]:
plot_tsne(tcell_df, n_plots=1)

In [70]:
# from sklearn.model_selection import train_test_split
# def normalize(x):
#     data = x.values
#     data = 2 * (data - np.nanmin(data, axis=0)) / (np.nanmax(data, axis=0) - np.nanmin(data, axis=0)) - 1
#     return pd.DataFrame(data=data, columns=x.columns, index=x.index)


# def load_data_basic(path, sample='sample1', batch_names=['batch1', 'batch2'], 
#                     panel=None, seed=42, n_cells_to_select=0, test_size=0.2):
#     """
#     Function to load data and split into 2 inputs with train and test sets
#     inputs:
#         path: path to the data file
#         sample: name of the sample to consider
#         batch_names: a list of batch names to split the data
#         n_cells_to_select: number of cells to select for quicker runs, if 0 then all cells are selected (min of the 2 batches)
#         test_size: proportion of the test set
#     outputs:
#         x1_train, x1_test: train and test sets form the first batch
#         x2_train, x2_test: train and test sets form the second batch
#     """
#     df = pd.read_parquet(path, engine='pyarrow')
#     if(panel is not None):
#         df = df.loc[df['metadata_panel'].str.startswith(panel),:]
#         # update batches names that are present in the panel
#         panel_batch_names = list(set(df.loc[:,'metadata_batch']))
#         if(len([x for x in batch_names if x not in panel_batch_names])):
#             batch_names = panel_batch_names
#     # remove columns with ann (bcs of merging the panels)
#     df = df.dropna(axis=1)
#     if('metadata_batch' in df.columns and 'metadata_sample' in df.columns):
#         x1 = df.loc[(df['metadata_batch']==batch_names[0]) & (df['metadata_sample']==sample),:].copy()
#         x2 = df.loc[(df['metadata_batch']==batch_names[1]) & (df['metadata_sample']==sample),:].copy()
#     else:
#         idx = df.index.get_values()
#         x1_idx = [x for x in idx if sample in x and batch_names[0] in x and sample+'0' not in x]
#         x1_idx = [i for (i,t) in enumerate(idx) if t in x1_idx]
#         x1 = df.loc[x1_idx, :].copy()
#         x2_idx = [x for x in idx if sample in x and batch_names[1] in x and sample+'0' not in x]
#         x2_idx = [i for (i,t) in enumerate(idx) if t in x2_idx]
#         x2 = df.loc[x2_idx, :].copy()
#     # remove metadata columns
#     selected_cols = [col for col in df.columns if "metadata" not in col]
#     x1 = x1.loc[:, selected_cols]
#     x2 = x2.loc[:, selected_cols]
#     if n_cells_to_select > 0:
#         n_cells_to_select = np.min([n_cells_to_select,x1.shape[0], x2.shape[0]])
#     else:
#         n_cells_to_select = np.min([x1.shape[0], x2.shape[0]])
#     cells_to_select = np.random.uniform(0, x1.shape[0], n_cells_to_select)
#     x1 = x1.iloc[cells_to_select, :]
#     cells_to_select = np.random.uniform(0, x2.shape[0], n_cells_to_select)
#     x2 = x2.iloc[cells_to_select, :]
#     x1 = normalize(x1)
#     x2 = normalize(x2)
#     x1_train, x1_test = train_test_split(x1, test_size=test_size, random_state=seed)
#     x2_train, x2_test = train_test_split(x2, test_size=test_size, random_state=seed)
#     return x1_train, x1_test, x2_train, x2_test
